In [1]:
from itertools import tee
from typing import List
def ngrams(sequence: List[str], n: int, min_length: int = 5):
    """
    Return the ngrams generated from a sequence of items, as an iterator.

    This is a modified version of nltk.util.ngrams.

    Parameters
    ----------
    sequence : List[Text]
        The sequence of items.
    n : int
        The length of each ngram.
    min_length : int, optional
        The minimum length of each ngram, by default 5

    Returns
    -------
    iterator
        The ngrams.

    Examples
    --------
    >>> list(ngrams(["a", "b", "c", "d"], 2, min_length=1))
    [('a', 'b'), ('b', 'c'), ('c', 'd')]
    >>> list(ngrams(["a", "b", "c", "d"], 2, min_length=5))
    []
    >>> list(ngrams(["a", "b"], 3, min_length=1))
    [('a', 'b')]
    """
    if len(sequence) < min_length:
        return []
    if len(sequence) < n:
        return [tuple(sequence)]
    iterables = tee(iter(sequence), n)
    for i, sub_iterable in enumerate(iterables):
        for _ in range(i):
            next(sub_iterable, None)
    return zip(*iterables)


In [2]:
import hashlib
import struct

def sha1_hash(data: bytes, d: int = 32) -> int:
    """
    Generate a d-bit hash value from the given data.

    Parameters
    ----------
    data : bytes
        The data to be hashed.
    d : int
        The number of bits of the hash value.

    Returns
    -------
    int
        The hash value.

    Examples
    --------
    >>> sha1_hash(b"hello world", 32)
    896314922
    >>> sha1_hash(b"hello world", 64)
    13028719972609469994
    >>> sha1_hash(b"hello world", 128)
    310522945683037930239412421226792791594
    """
    if d == 32:
        return struct.unpack("<I", hashlib.sha1(data, usedforsecurity=False).digest()[:4])[0]
    if d == 64:
        return struct.unpack("<Q", hashlib.sha1(data, usedforsecurity=False).digest()[:8])[0]
    # struct is faster but does not support arbitrary bit lengths
    return int.from_bytes(hashlib.sha1(data, usedforsecurity=False).digest()[: d // 8], byteorder="little")


In [3]:
import re
import numpy as np
SEED = 42
RNG = np.random.RandomState(SEED)
NON_ALPHA = re.compile(r"\W", re.UNICODE)
SIGNATURE_COLUMN = "__signatures__"
content = "But I must explain to you how all this mistaken idea of denouncing pleasure and praising pain was born and I will give you a complete account of the system, and expound the actual teachings of the great explorer of the truth, the master-builder of human happiness. No one rejects, dislikes, or avoids pleasure itself, because it is pleasure, but because those who do not know how to pursue pleasure rationally encounter consequences that are extremely painful. Nor again is there anyone who loves or pursues or desires to obtain pain of itself, because it is pain, but because occasionally circumstances occur in which toil and pain can procure him some great pleasure. To take a trivial example, which of us ever undertakes laborious physical exercise, except to obtain some advantage from it? But who has any right to find fault with a man who chooses to enjoy a pleasure that has no annoying consequences, or one who avoids a pain that produces no resultant pleasure?"

tokens: set[bytes] = {
        bytes(" ".join(t).lower(), "utf-8") for t in ngrams(NON_ALPHA.split(content.lower()), 3, 5)
    }

hashvalues: np.ndarray = np.array([sha1_hash(token) for token in tokens], dtype=np.uint64).reshape(len(tokens), 1)


In [4]:
dtype, max_hash, modulo_prime =  (np.uint64, np.uint32((1 << 32) - 1), np.uint64((1 << 61) - 1))

In [5]:
PERMUTATIONS: tuple[np.ndarray, np.ndarray] = (
        RNG.randint(
            1, modulo_prime, size=(200,), dtype=dtype
        ),  # a is a multiplier so should not be 0
        RNG.randint(0, modulo_prime, size=(200,), dtype=dtype),  # b
    )

In [7]:
def sample():
    sample = np.array([[1,2,3]])
    a = np.array([1,2,3,4,5])
    b = np.array([1,3,5,6,7])
    print(a.shape)
    print(b.shape)
    print(f"{sample.T * a}")
    
    print(f"{sample.T * a + b}")

In [8]:
b = np.array([1,1,1,1,1])

In [9]:
a,b = PERMUTATIONS
hashvalues_after_ab = (hashvalues * a + b) % modulo_prime & max_hash

In [11]:
hashvalues_after_ab.shape

(177, 200)

In [12]:
masks: np.ndarray = np.full(shape=200, dtype=dtype, fill_value=max_hash)

In [29]:
hashvalues_after_ab.shape

(177, 200)

In [27]:
hashvalues_after_mask = np.vstack([hashvalues_after_ab, masks]).min(axis=0)

In [28]:
hashvalues_after_mask.shape 

(200,)

In [21]:
from text_dedup.utils.analysis import optimal_param

B, R = optimal_param(0.5, 200,0.5,0.5)

hashranges = [(i * R, (i + 1) * R) for i in range(B)]

Hs: list[bytes] = [bytes(hashvalues[start:end].byteswap().data) for start, end in hashranges]

In [32]:
start, end = hashranges[0]

In [35]:
hashvalues[start:end]

array([[2169222163],
       [ 589797548],
       [ 657804509],
       [3661243181],
       [2206766972],
       [2577906337]], dtype=uint64)

In [38]:
bytes(hashvalues[start:end].byteswap().data)

b"\x00\x00\x00\x00\x81K\xb4\x13\x00\x00\x00\x00#'\x98\xac\x00\x00\x00\x00'5L\xdd\x00\x00\x00\x00\xda:#-\x00\x00\x00\x00\x83\x88\x97|\x00\x00\x00\x00\x99\xa7\xba\xa1"